
# **数据挖掘——Home Credit Default Risk**

Authors：李林（3120220938）、杨洋（3220211141）、敬甲男（3220221052）、李翰杰（3120220936）

github地址：https://github.com/leealim/kaggle-Home-Credit-Default-Risk

---

## 数据预处理——缺失值处理

共八张表，逐个进行处理：
- application_{train|test}.csv:客户申请表
- bureau.csv/bureau_balance.csv:客户历史借款记录
- POS_CASH_balance.csv:客户POS和现金贷款历史
- credit_card_balance.csv:客户信用卡的snapshot历史
- previous_application.csv:客户历史申请记录
- installments_payments.csv:客户信用卡还款记录

---


In [3]:
# 引入本部分所需要的包，并定义需要的值和函数

import pandas as pd
import os

source_dir="..\\"
if not os.path.exists(source_dir):
    raise Exception('请补充数据集！')
result_dir="..\\data\\miss_value_handling"

app_tr_path = source_dir+"\\application_train.csv"
app_te_path = source_dir+"\\application_test.csv"
bur_path = source_dir+"\\bureau.csv"
bur_bal_path = source_dir+"\\bureau_balance.csv"
pos_path = source_dir+"\\POS_CASH_balance.csv"
cre_path = source_dir+"\\credit_card_balance.csv"
pre_path = source_dir+"\\previous_application.csv"
ins_path = source_dir+"\\installments_payments.csv"
hom_path = "..\\data\\home-credit-default-risk\\HomeCredit_columns_description.csv"  # 列描述表
hom = pd.read_csv(hom_path)



if not os.path.exists(result_dir):
    os.makedirs(result_dir)

def missing_values_table(df, table_name):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(
        columns={0: 'Missing Values',
                 1: '% of Total Values'})
    mis_val_table = mis_val_table.sort_values(
        '% of Total Values', ascending=False).round(1)
    miss_num = (mis_val_table["Missing Values"] != 0).sum()
    print("Total " + str(miss_num) + " columns missing values")
    mis_val_table = mis_val_table.drop(index=mis_val_table[miss_num:].index)
    mis_val_table = mis_val_table.merge(
        hom, how="left", left_index=True, right_on='Row')
    mis_val_table = mis_val_table.drop(columns=['Unnamed: 0'])
    mis_val_table = mis_val_table.drop(
        index=mis_val_table.loc[mis_val_table["Table"] != table_name].index)
    mis_val_table = mis_val_table.reindex(
        columns=["Row", "Description", "Special", "Missing Values", "% of Total Values"])
    mis_val_table = mis_val_table.reset_index(drop=True)
    mis_val_table = mis_val_table.merge(
        df.describe().T, how="left", left_on="Row", right_index=True)
    return mis_val_table



### 1. **application_{train|test}.csv**

In [ ]:
# 查看训练数据的基本数据特征

app_tr = pd.read_csv(app_tr_path)
app_tr.describe()


In [ ]:
# 缺失值分析

t = missing_values_table(app_tr, "application_{train|test}.csv")
pd.set_option("display.max_rows", 20)
pd.set_option('max_colwidth',40)
t


In [ ]:
#获取较小的缺失值列信息

t_small=t.loc[t["% of Total Values"]<3]
t_small


In [ ]:
#获取较大的缺失值列信息

t_large=t.loc[t["% of Total Values"]>3]
t_large

可以发现，残缺值数量差距很大，对于小于百分之一的残缺值，我们采取删去对应行的措施。


In [ ]:
#删去部分行

app_tr.dropna(subset=t_small["Row"],
          axis=0, # axis=0表示删除行；
          how='any', # how=any表示若列name、age中，任意一个出现空值，就删掉该行
          inplace=True # inplace=True表示在原df上进行修改；
          )
app_tr = app_tr.reset_index(drop=True)
app_tr

对于残缺值比较大的行，我们逐一进行研究处理。首先，对每个特征融入统计数据。

In [ ]:
t = missing_values_table(app_tr, "application_{train|test}.csv")
pd.set_option("display.max_rows", 400)
pd.set_option('max_colwidth',400)
t




可以发现还存在一些存在大量残缺值的分类数据。对于这些数据，在转化为数值数据时，多转化一个类别。
另外，这里面有着大量缺失的房产数据，可以简化这些特征的拥有总和为一个特征，表明拥有房产的可信度。



In [ ]:
#填补分类数据缺失值

temp=t.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp==True].index
rows=t.loc[temp.loc[temp==True].index].Row

for col in rows: 
    app_tr[str(col)] = app_tr[str(col)].fillna(value="MyNull")
    
app_tr[str(rows.iloc[0])]

In [ ]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
pd.set_option("display.max_rows", 20)
pd.set_option('max_colwidth',40)
t

In [ ]:
#重新理解房产数据,暂时存储删除列和test表合并删除，并将删除列的空值填补任意值。
print(app_tr.shape)
t_house=t.loc[t["Description"]=="Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp=app_tr.loc[:,t_house["Row"].tolist()].isnull().sum(axis=1)
app_tr["MY_HOUSE_OWN"]=temp
app_tr_drop_list=t_house["Row"].tolist()
for col in app_tr_drop_list: 
    app_tr[col] = app_tr[col].fillna(value=0)
app_tr

In [ ]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

目前，还有9列缺失值，可以发现OWN_CAR_AGE是车辆拥有年限数据，缺失值可以置零   
可以看到EXT_SOURCE_1和EXT_SOURCE_3比较平滑，直接用平均值代替  
AMT_REQ_CREDIT_BUREAU的一列数据用0填补，缺失值可能说明这些客户并未有对应的enquiries

In [ ]:
#处理OWN_CAR_AGE

app_tr["OWN_CAR_AGE"] = app_tr["OWN_CAR_AGE"].fillna(value=0)
app_tr["OWN_CAR_AGE"]


In [ ]:
#处理EXT_SOURCE_1和EXT_SOURCE_3

tr_1_mean_val = app_tr["EXT_SOURCE_1"].mean()
app_tr["EXT_SOURCE_1"].fillna(tr_1_mean_val, inplace=True)
tr_2_mean_val = app_tr["EXT_SOURCE_3"].mean()
app_tr["EXT_SOURCE_3"].fillna(tr_2_mean_val, inplace=True)


In [ ]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

In [ ]:
# 处理 AMT_REQ_CREDIT_BUREAU

for s in t["Row"].tolist():
    app_tr[s].fillna(0, inplace=True)

In [ ]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

至此，主表缺失值处理完毕。
此外，也对app_te表进行处理


In [ ]:
app_te = pd.read_csv(app_te_path)

t = missing_values_table(app_te, "application_{train|test}.csv")
t_small = t.loc[t["% of Total Values"] < 3]
t_large = t.loc[t["% of Total Values"] > 3]
for c in t_small["Row"].tolist():
    if app_tr[c].dtype == object:
        cmax = max(app_tr[c])
        app_te[c].fillna(cmax,
                  inplace=True  # inplace=True表示在原df上进行修改；
                  )
    else:
        mean_num = app_tr[c].mean()
        app_te[c].fillna(mean_num,
                  inplace=True  # inplace=True表示在原df上进行修改；
                  )
    

t = missing_values_table(app_te, "application_{train|test}.csv")
temp = t.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp == True].index
rows = t.loc[temp.loc[temp == True].index].Row
for col in rows:
    app_te[str(col)] = app_te[str(col)].fillna(value="MyNull")

t = missing_values_table(app_te, "application_{train|test}.csv")
t_house = t.loc[t["Description"] ==
                "Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp = app_te.loc[:, t_house["Row"].tolist()].isnull().sum(axis=1)
app_te["MY_HOUSE_OWN"] = temp
app_te_drop_list = t_house["Row"].tolist()
for col in app_te_drop_list:
    app_te[col] = app_te[col].fillna(value=0)

t = missing_values_table(app_te, "application_{train|test}.csv")
app_te.loc[:,t_small["Row"].tolist()]


In [ ]:
# 可以发现和train表是一致的

app_te["OWN_CAR_AGE"] = app_te["OWN_CAR_AGE"].fillna(value=0)
app_te["EXT_SOURCE_1"].fillna(tr_1_mean_val, inplace=True)
app_te["EXT_SOURCE_3"].fillna(tr_2_mean_val, inplace=True)
t=missing_values_table(app_te,"application_{train|test}.csv")
for s in t["Row"].tolist():
    app_te[s].fillna(0, inplace=True)
t=missing_values_table(app_te,"application_{train|test}.csv")
t

In [ ]:
# 最后把准备删除的行给删除了

app_tr.drop(columns=list(set(app_tr_drop_list+app_te_drop_list)),inplace=True)
app_te.drop(columns=list(set(app_tr_drop_list+app_te_drop_list)),inplace=True)
print(app_tr.shape)
print(app_te.shape)

In [ ]:
# 结果保存

app_tr.to_csv(result_dir+"\\application_train.csv",index=False)
app_te.to_csv(result_dir+"\\application_test.csv",index=False)


### 2. **previous_application.csv**

In [ ]:
pre = pd.read_csv(pre_path)

In [ ]:
# 结果保存

pre.to_csv(result_dir+"\\previous_application.csv",index=False)